In [1]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
data_one = []
reviewdata_df = []

In [3]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [4]:
df = pd.DataFrame(data_one)

In [5]:
df.shape

(556303, 8)

In [6]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [7]:
df_user = pd.DataFrame(data_main)

In [8]:
def user_with_friends(x):
    if len(x['friends']) != 0:
        return x

In [9]:
df_user_with_friends = df_user.apply(user_with_friends, axis=1)

In [10]:
df_user_with_friends = df_user_with_friends.dropna()

In [11]:
df_user.shape


(552339, 11)

In [12]:
df_user_with_friends.shape

(249440, 11)

In [13]:
merged_df = pd.merge(df, df_user_with_friends, on='user_id', how='inner')

In [14]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,{},[],2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",2013-05


In [15]:
merged_df.shape

(394815, 18)

In [16]:
merged_df.stars.value_counts(ascending=True)

2     36085
1     38347
3     63586
4    124241
5    132556
Name: stars, dtype: int64

In [17]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [18]:
df_business = pd.DataFrame(data_business)

In [19]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [20]:
business_merged_df.shape

(394815, 32)

In [21]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state',
                         'open', 'hours', 'attributes'], axis=1, inplace=True)

In [22]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,categories,name_y,review_count_y,stars_y,type
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",[Nightlife],Clancy's Pub,4,3.5,business


In [24]:
%timeit business_sorted_df = business_merged_df.sort_index(by=['business_id'], ascending=[True])

1 loops, best of 3: 1.34 s per loop


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:257: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [55]:
for row in business_merged_df.itertuples():
    if row[1] == 'UsFtqoBl7naz8AVUBZMjQQ':
        print row[16]

[u'Nightlife']
[u'Nightlife']
[u'Nightlife']


In [76]:
business_merged_df = business_merged_df.loc[business_merged_df.categories.apply(lambda cat: 'Food' in cat or 'Restaurants' in cat)]

In [79]:
business_merged_df.shape

(252836, 21)

In [81]:
grouped = business_merged_df.groupby(business_merged_df['business_id'])

In [82]:
def look_for_user_ratings1(group, biz_id):
    ur = []
    for row in group.itertuples():
        ur.append({
                "user_id": row[7],
                "user_rating": row[4],
            })
    return ur


In [83]:
business_user1 = {}
li = []
for business_id, group in grouped:
    business_user1.update({
        business_id: look_for_user_ratings1(group, business_id)
    })

In [95]:
print business_user1.get('PSAK5sUss8yiN7Q_72Zlxw')

[{'user_rating': 5, 'user_id': u'F6QsMoJdvtohlbnST-fDyQ'}, {'user_rating': 3, 'user_id': u'wuCszpVPMIMyOCualnZtaw'}, {'user_rating': 2, 'user_id': u'pQ6ij6cOBRB3yJGHbVZHUQ'}, {'user_rating': 4, 'user_id': u'MNfOCJZsbtMbnDlvJ1LKeQ'}, {'user_rating': 5, 'user_id': u'_LsbT_uW0UE6rhsWiAIrsg'}, {'user_rating': 3, 'user_id': u'ND95DgvN7ba8MabfXE3lsw'}, {'user_rating': 3, 'user_id': u'qeljTFFJaZE-HJ6RKx0iFg'}, {'user_rating': 4, 'user_id': u'HxZ6tI_7G1ssxlZGV6dByA'}]


In [93]:
for row in business_merged_df.itertuples():
    if row[1] == "PSAK5sUss8yiN7Q_72Zlxw":
        print "user id" + str(row[7]) + " rated " + str(row[4])

user idF6QsMoJdvtohlbnST-fDyQ rated 5
user idwuCszpVPMIMyOCualnZtaw rated 3
user idpQ6ij6cOBRB3yJGHbVZHUQ rated 2
user idMNfOCJZsbtMbnDlvJ1LKeQ rated 4
user id_LsbT_uW0UE6rhsWiAIrsg rated 5
user idND95DgvN7ba8MabfXE3lsw rated 3
user idqeljTFFJaZE-HJ6RKx0iFg rated 3
user idHxZ6tI_7G1ssxlZGV6dByA rated 4


In [84]:
##This is the 2nd hashmap

user_friends = {}

for row in df_user_with_friends.itertuples():
    user_id = row[9]
#    print user_id
    user_friends.update({
        user_id: row[5],
    })

In [103]:
business_merged_df.head(n=10)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,...,name_x,review_count_x,type_y,votes_y,categories,name_y,review_count_y,stars_y,type,desired_category
3,mVHrayjG3uZ_RLHkLj-AMg,2013-10-20,KAkcn7oQP1xX8KsZ-XmktA,4,This place was very good. I found out about Em...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 1, u'cool': 0}",3.84,2,...,Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
4,mVHrayjG3uZ_RLHkLj-AMg,2013-03-15,jVVv_DA5mCDB6mediuwHAw,5,Can't miss stop for the best Fish Sandwich in ...,review,m1FpV3EAeggaAdfPx0hBRQ,"{u'funny': 0, u'useful': 0, u'cool': 0}",4.00,1,...,Fred,4,user,"{u'funny': 1, u'useful': 1, u'cool': 0}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
5,mVHrayjG3uZ_RLHkLj-AMg,2013-03-30,3Es8GsjkssusYgeU6_ZVpQ,5,This place should have a lot more reviews - bu...,review,8fApIAMHn2MZJFUiCQto5Q,"{u'funny': 0, u'useful': 3, u'cool': 1}",3.40,0,...,Bennett,34,user,"{u'funny': 16, u'useful': 42, u'cool': 10}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
6,mVHrayjG3uZ_RLHkLj-AMg,2013-11-07,BZNJkkP0bXnwQ2-sCqat2Q,5,"Old school.....traditional ""mom 'n pop"" qualit...",review,6wvlM5L4_EroGXbnb_92xQ,"{u'funny': 0, u'useful': 0, u'cool': 0}",2.92,0,...,David,13,user,"{u'funny': 15, u'useful': 35, u'cool': 6}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
7,mVHrayjG3uZ_RLHkLj-AMg,2014-11-03,uf61rPucuICXhSPXlZ1hIQ,5,"After a morning of Thrift Store hunting, a fri...",review,tAKjY3bQXH51msJbOHYPmQ,"{u'funny': 0, u'useful': 3, u'cool': 1}",3.52,8,...,William,118,user,"{u'funny': 195, u'useful': 457, u'cool': 162}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
8,mVHrayjG3uZ_RLHkLj-AMg,2014-12-18,xY2gjy49dnpQSB2RTZzqCw,4,"This is a hidden gem, no really. It took us fo...",review,Kq8-FUG7d_MT2qRNiNBJnA,"{u'funny': 0, u'useful': 0, u'cool': 0}",4.26,2,...,Gloria,19,user,"{u'funny': 0, u'useful': 2, u'cool': 1}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
9,mVHrayjG3uZ_RLHkLj-AMg,2015-03-06,1W1vq7D1ofwFKbCA_GWu_A,5,"From reading the other reviews, I see this pla...",review,a0ULPTNi05w-BNVovVUz2w,"{u'funny': 1, u'useful': 2, u'cool': 4}",3.92,3,...,Sean,13,user,"{u'funny': 12, u'useful': 25, u'cool': 25}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
10,mVHrayjG3uZ_RLHkLj-AMg,2015-03-08,6A5nASW8_XRwDxV29cWGUQ,4,Another divey place we have been going to for ...,review,-ke0O45_B6STtEqd5s_ANw,"{u'funny': 0, u'useful': 0, u'cool': 0}",4.24,0,...,Arla,17,user,"{u'funny': 3, u'useful': 18, u'cool': 5}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
11,mVHrayjG3uZ_RLHkLj-AMg,2015-12-04,MKEs8po2_DFt9msCp8w68A,5,"We have been coming here for over 30 years, co...",review,LQUy7D5Bzb2WutEXhVg2Tw,"{u'funny': 0, u'useful': 0, u'cool': 0}",5.00,0,...,Jennifer,1,user,"{u'funny': 0, u'useful': 0, u'cool': 0}","[Bars, American (New), Nightlife, Lounges, Res...",Emil's Lounge,20,5.0,business,foodie
12,k1A8pNspBPinGDivxkuDSA,2013-11-08,3PqxM6TD-MHsofAIqOoMKw,4,If you love Pittsburgh sports & classic Pittsb...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,...,Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}","[Bars, American (Traditional), Nightlife, Rest...",Clark Bar & Grill,11,3.5,business,foodie


In [85]:
def look_for_business_ratings(business_user_dict, business_id, user_ids):
    ratings = business_user_dict.get(business_id)
    sum_of_stars = 0
    count = 0
    business_reviewers = [li['user_id'] for li in ratings]
    for i, uid in enumerate(business_reviewers):
        for user in user_ids:
            if uid == user:
                count += 1
                sum_of_stars += ratings[i]['user_rating']
    avg = float(sum_of_stars/float(count))
    return avg

In [99]:
new_df = []
for row in business_merged_df.itertuples():
    business_id = row[1]
    user_id = row[7]
    ratings = business_user1.get(business_id)
    
    friends = user_friends.get(user_id)
    business_reviewers = [li['user_id'] for li in ratings]

    common_reviewers = list(set(friends) & set(business_reviewers))
    if len(common_reviewers) == 0:
        continue
    else:
        friends_avg = look_for_business_ratings(business_user1, business_id, common_reviewers)
        
    new_df.append({
            "Business_id": business_id,
            "User_id": user_id,
            "friends_avg_rating": friends_avg,
            "user_rating": row[4],
            "friends_who_rated": len(common_reviewers),
            "friends": len(friends),
            "no_of_reviewers": len(business_reviewers),
            "business_rating": row[19]
        })

final_df = pd.DataFrame(new_df)

In [100]:
 final_df.shape

(90054, 8)

In [104]:
corr_df = final_df[['friends_avg_rating', 'user_rating']]

In [105]:
corr_df.corr()

,friends_avg_rating,user_rating
friends_avg_rating,1.000000,0.321905
user_rating,0.321905,1.000000


In [119]:
corr_df1 = final_df[['business_rating', 'friends_avg_rating']]

In [120]:
corr_df1.corr()

,business_rating,friends_avg_rating
business_rating,1.00000,0.47143
friends_avg_rating,0.47143,1.00000


In [101]:
final_df.head(n=10)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,5,2,5.000000,1,9,5
1,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,5,31,5.000000,1,9,5
2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4,1444,4.000000,22,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4,45,4.000000,3,59,1
4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4,6,4.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4,149,3.333333,3,59,3
6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4,2,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4,64,4.333333,3,59,5
8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4,5,5.000000,1,59,5
9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4,16,5.000000,1,59,4


In [102]:
final_df.business_rating.value_counts(ascending=True)

1.0        4
1.5       76
5.0      107
2.0      615
2.5     2652
3.0     8736
4.5     9919
3.5    26768
4.0    41177
Name: business_rating, dtype: int64

In [106]:
final_df_no_duplicates = final_df.drop_duplicates(keep='first')

In [107]:
final_df_no_duplicates.shape

(87745, 8)

In [108]:
final_df_no_duplicates.Business_id.unique().shape

(3988L,)

In [109]:
X = final_df_no_duplicates.user_rating
Y = final_df_no_duplicates.friends_avg_rating

In [110]:
X = X.reshape(len(X), 1)
Y = Y.reshape(len(Y), 1)

In [111]:
clf = LinearRegression()
clf = clf.fit(X, Y)

In [112]:
r_sqr = clf.score(X, Y)
y_pred = clf.predict(X)
intercept = clf.intercept_
coefficient = clf.coef_

In [113]:
r_sqr

0.10277195085274393

In [114]:
coefficient

array([[ 0.25124832]])

In [115]:
final_df_no_duplicates_4 = final_df_no_duplicates[(final_df_no_duplicates.business_rating == 4)]

In [122]:
final_df_no_duplicates_4.reset_index(drop=True)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4,1444,4.000000,22,59,5
1,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4,45,4.000000,3,59,1
2,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4,6,4.000000,1,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4,149,3.333333,3,59,3
4,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4,2,5.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4,64,4.333333,3,59,5
6,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4,5,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4,16,5.000000,1,59,4
8,YssP-_6iP0SLBsqUQGs6rQ,iTmWHtltCtk0Gm55AOxrUA,4,93,5.000000,2,59,4
9,YssP-_6iP0SLBsqUQGs6rQ,sPQ10u9ZsrCVCKhQylFxgQ,4,89,4.428571,4,59,2


In [ ]:
#this is the one making the most sense
plt.axhline(y=4.0)
#plt.scatter(final_df_no_duplicates_4.index, final_df_no_duplicates_4.user_rating, c='b', label='user ratings')
#plt.scatter(final_df_no_duplicates_4.index, final_df_no_duplicates_4.friends_avg_rating, c='r', label='friends_avg')
plt.scatter(final_df_no_duplicates_4.user_rating, final_df_no_duplicates_4.friends_avg_rating, c='b', label='user ratings')


In [128]:
plt.show()